In [1]:
import os

from astropy.io import fits
from astropy import units as u
from astropy.coordinates import Angle
from astroquery.vizier import Vizier

import pprint

import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', '{:.6f}'.format)

In [2]:
def query_variable_star(star_name):
    v = Vizier(columns=['*'])
    v.ROW_LIMIT = 5  # Limit the number of rows to keep the output manageable

    # Query the VSX (Variable Star Index) catalog
    result = v.query_object(star_name, catalog=["B/vsx/vsx"])

    # Check if results were returned
    if result and len(result) > 0:
        vsx_table = result[0]  # Assuming the star is found in the first returned table
        
        name = vsx_table['Name'][0]
        
        ra_angle = Angle(vsx_table['RAJ2000'][0]/15.0, u.hour)
        ra = ra_angle.to_string(unit=u.hour, sep=' ', precision=2, pad=True)
        
        dec_angle = Angle(vsx_table['DEJ2000'][0], u.deg)
        dec = dec_angle.to_string(unit=u.degree, sep=' ', precision=2, alwayssign=True, pad=True)        
        
        period = vsx_table['Period'][0] if 'Period' in vsx_table.colnames else "N/A"
        mag_max = vsx_table['max'][0] if 'max' in vsx_table.colnames else "N/A"
        mag_min = vsx_table['min'][0] if 'min' in vsx_table.colnames else "N/A"

        var_type = vsx_table['Type'][0] if 'Type' in vsx_table.colnames else "N/A"
        spectral_type = vsx_table['Sp'][0] if 'Sp' in vsx_table.colnames else "N/A"

        # Convert the table to a dictionary with custom key names
        custom_dict = {
            'name': name,
            'ra': ra,
            'dec': dec,
            'period': period,
            'mag_max': mag_max,
            'mag_min': mag_min,
            'var_type': var_type,
            'spectral_type': spectral_type
        }

        return custom_dict

    else:
        print("No results found for", star_name)
        return {}

def dms_to_decimal(degrees, minutes, seconds, direction):
    return Angle(f"{degrees}d{minutes}m{seconds}s{direction}").value

def dump_header(filename):
    with fits.open(filename) as hdul:
        for hdu in hdul:
            print("Header of HDU:", hdu.name)
            for key, value in hdu.header.items():
                print(f"{key}: {value}")
            print("\n")

def update_header(filename, values_to_add, keys_to_remove):
    with fits.open(filename, mode='update') as hdul:

        for key, value in values_to_add.items():
            hdul[0].header[key] = value

        for key in keys_to_remove:
            if key in hdul[0].header:
                del hdul[0].header[key]
        hdul.flush()

In [3]:
star = 'RS Boo'
directory = '/Users/jwatts/astrophotography/Photometry/4.20/Light/RSBoo/'

obj = star
star_data = query_variable_star(obj)
objectra = star_data.get('ra', 0.0)
objectdec = star_data.get('dec', 0.0)

latitude = dms_to_decimal(19, 42, 29.97, 'N')
longitude = dms_to_decimal(155, 58, 45.07, 'W')
altitude = (1535.4 * u.imperial.ft).to(u.m).value

print (f"RA: {objectra}  ({Angle(objectra, u.hour).to(u.deg).value})")
print (f"Dec: {objectdec}  ({Angle(objectdec, u.deg).value})")
print("===================")
pprint.pprint(star_data)

RA: 14 33 33.21  (218.388375)
Dec: +31 45 16.60  (31.75461111111111)
{'dec': '+31 45 16.60',
 'mag_max': 9.69,
 'mag_min': 10.84,
 'name': 'RS Boo',
 'period': 0.37733896,
 'ra': '14 33 33.21',
 'spectral_type': 'A7-F5',
 'var_type': 'RRAB'}


In [4]:
values_to_add = {
    'SITEELEV': (altitude, 'Site elevation in meters'),
    'SITELAT': (latitude, 'Site latitude in degrees'),
    'SITELONG': (longitude, 'Site longitude in degrees'),
    'OBJECT': (obj, 'Target object name'),
    'OBJCTRA': (objectra, '[hms J2000] Target right ascension'),
    'OBJCTDEC': (objectdec, '[dms +N J2000] Target declination')
}

keys_to_remove = ['RA', 'DEC']

for filename in os.listdir(directory):
    if filename.endswith('.fit'):
        fits_file = os.path.join(directory, filename)
        print(f"FITS FILE: {fits_file}")
        #update_header(fits_file, values_to_add, keys_to_remove)
        dump_header(fits_file)


FITS FILE: /Users/jwatts/astrophotography/Photometry/4.20/Light/RSBoo/Light_RSBoo_40.0s_Bin1_20240420-050745_0001.fit
Header of HDU: PRIMARY
SIMPLE: True
BITPIX: 16
NAXIS: 2
NAXIS1: 4144
NAXIS2: 2822
EXTEND: True
COMMENT:   FITS (Flexible Image Transport System) format is defined in 'Astronomy
COMMENT:   and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H
BZERO: 32768
BSCALE: 1
CREATOR: ZWO ASIAIR Plus
OFFSET: 30
XORGSUBF: 0
YORGSUBF: 0
FOCALLEN: 1286
SET-TEMP: -20
EGAIN: 3.99000000953674
XBINNING: 1
YBINNING: 1
CCDXBIN: 1
CCDYBIN: 1
XPIXSZ: 4.63000011444092
YPIXSZ: 4.63000011444092
IMAGETYP: Light
EXPOSURE: 40.0
EXPTIME: 40.0
CCD-TEMP: -19.7999992370605
DATE-OBS: 2024-04-20T15:07:04.851534
INSTRUME: ZWO ASI294MC Pro
BAYERPAT: RGGB
GAIN: 0
TELESCOP: Celestron AVX/CGE/CGEM/CGX
CTYPE1: RA---TAN-SIP
CTYPE2: DEC--TAN-SIP
CRVAL1: 218.130649591
CRVAL2: 31.6298191053
CRPIX1: 2569.71447754
CRPIX2: 270.315404892
CD1_1: 1.6783289331825e-05
CD1_2: 0.00020637050712525
CD2_1: -0.0